In [23]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
messages = pd.read_csv('smsspamcollection/SMSSpamCollection', sep='\t', names=['label', 'messages'])

In [25]:
messages.shape

(5572, 2)

### Data Cleaning

In [26]:
# ps = PorterStemmer()
ps = SnowballStemmer('english')
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['messages'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

**Creating a bag of words**

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000)   # consider top 2500 words ordered by term frequency
x = cv.fit_transform(corpus)

In [32]:
y = pd.get_dummies(messages['label'])
y=y.iloc[:,1].values

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [34]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [14]:
y_pred = spam_detect_model.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [16]:
# accuracy was 98.47% in Lancester Stemmer
accuracy_score(y_pred, y_test)

0.9856502242152466

In [17]:
confusion_matrix(y_test, y_pred)

array([[947,   8],
       [  8, 152]], dtype=int64)

In [18]:
def predict_me(message):
    message = re.sub('[^a-zA-Z]', ' ', message)
    message = message.lower()
    message = message.split()
    
    message_cleaned = [ps.stem(word) for word in message if not word in stopwords.words('english')]
    message_cleaned = ' '.join(message_cleaned)

    message_vect = cv.transform([message_cleaned]).toarray()  
    pred = spam_detect_model.predict(message_vect)
    if pred == 1:
        print("Spam")
    else:
        print("Not Spam")
    return 
    

In [37]:
strg = "Call me when you need"
strg_vec = cv.transform([strg]).toarray()
spam_detect_model.predict(strg_vec)

array([0], dtype=uint8)

In [20]:
predict_me(strg)

Not Spam


In [22]:
x = input("Enter any message to check as spam or not spam")
predict_me(x)

Enter any message to check as spam or not spamWe are inviting you to our Flipkart Big Fashion Sale, from 9th July to 15th July
Not Spam
